# Building Model For Predicting Bike Availability

# Loading data and merging into one data frame

In [37]:
import pandas as pd

stations = pd.read_csv("static/csv/stations.csv")
availability = pd.read_csv("static/csv/availability.csv")
weather = pd.read_csv("static/csv/current_weather.csv")

df = availability.merge(weather[["weather_desc", "temp", "visibility", "wind_speed", "wind_deg", "timestamp"]], on = ["timestamp"], how = "left")
df = df.merge(stations[["bike_stands", "number"]], on = ["number"], how = "left")
df.head()

,number,available_bikes,available_bike_stands,status,last_update,timestamp,weather_desc,temp,visibility,wind_speed,wind_deg,bike_stands
0,42,30,0,OPEN,1677185535000,1677185702,NaN,NaN,NaN,NaN,NaN,30.0
1,30,4,16,OPEN,1677185621000,1677185702,NaN,NaN,NaN,NaN,NaN,20.0
2,54,5,28,OPEN,1677185522000,1677185702,NaN,NaN,NaN,NaN,NaN,33.0
3,108,13,22,OPEN,1677185161000,1677185702,NaN,NaN,NaN,NaN,NaN,35.0
4,20,6,24,OPEN,1677185529000,1677185702,NaN,NaN,NaN,NaN,NaN,30.0


## Cleaning data frame

## Columns - dropping and renaming

In [38]:
df.drop(["status", "last_update"], axis = 1, inplace = True)
df.rename(columns = {"bike_stands": "total_stands", "available_bike_stands": "available_stands", "number": "station_num"}, inplace = True)
df.head()

,station_num,available_bikes,available_stands,timestamp,weather_desc,temp,visibility,wind_speed,wind_deg,total_stands
0,42,30,0,1677185702,NaN,NaN,NaN,NaN,NaN,30.0
1,30,4,16,1677185702,NaN,NaN,NaN,NaN,NaN,20.0
2,54,5,28,1677185702,NaN,NaN,NaN,NaN,NaN,33.0
3,108,13,22,1677185702,NaN,NaN,NaN,NaN,NaN,35.0
4,20,6,24,1677185702,NaN,NaN,NaN,NaN,NaN,30.0


## Rows - dropping rows with no weather data

In [39]:
df.dropna(inplace = True)
df.head()

,station_num,available_bikes,available_stands,timestamp,weather_desc,temp,visibility,wind_speed,wind_deg,total_stands
336580,42,16,14,1678113903,light rain,278.81,10000.0,3.09,290.0,30.0
336581,30,0,20,1678113903,light rain,278.81,10000.0,3.09,290.0,20.0
336582,54,17,16,1678113903,light rain,278.81,10000.0,3.09,290.0,33.0
336583,108,14,21,1678113903,light rain,278.81,10000.0,3.09,290.0,35.0
336584,20,22,8,1678113903,light rain,278.81,10000.0,3.09,290.0,30.0


## Date and Time - changing Unix timestamp

In [40]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit = "s")
df["month"] = df['timestamp'].dt.strftime('%B')
df["day"] = df['timestamp'].dt.day_name()
df["time_period"] = df['timestamp'].dt.strftime('%H') + '-' + (df['timestamp'].dt.hour + 1).astype(str)

df.drop("timestamp", axis = 1, inplace = True)
df.head()

,station_num,available_bikes,available_stands,weather_desc,temp,visibility,wind_speed,wind_deg,total_stands,month,day,time
336580,42,16,14,light rain,278.81,10000.0,3.09,290.0,30.0,March,Monday,14-15
336581,30,0,20,light rain,278.81,10000.0,3.09,290.0,20.0,March,Monday,14-15
336582,54,17,16,light rain,278.81,10000.0,3.09,290.0,33.0,March,Monday,14-15
336583,108,14,21,light rain,278.81,10000.0,3.09,290.0,35.0,March,Monday,14-15
336584,20,22,8,light rain,278.81,10000.0,3.09,290.0,30.0,March,Monday,14-15


## Reindexing columns

In [41]:
column_order = ["month", "day", "time_period", "station_num", "total_stands", "available_bikes", "available_stands", "temp", "weather_desc", "visibility", "wind_speed", "wind_deg"]
df = df.reindex(columns = column_order)
df.head()

,month,day,time,station_num,total_stands,available_bikes,available_stands,temp,weather_desc,visibility,wind_speed,wind_deg
336580,March,Monday,14-15,42,30.0,16,14,278.81,light rain,10000.0,3.09,290.0
336581,March,Monday,14-15,30,20.0,0,20,278.81,light rain,10000.0,3.09,290.0
336582,March,Monday,14-15,54,33.0,17,16,278.81,light rain,10000.0,3.09,290.0
336583,March,Monday,14-15,108,35.0,14,21,278.81,light rain,10000.0,3.09,290.0
336584,March,Monday,14-15,20,30.0,22,8,278.81,light rain,10000.0,3.09,290.0
